In [17]:
import cv2
import os
import random
import numpy as np
import smartcrop
from PIL import Image
from pprint import pprint
import shutil

In [18]:
def translate_image(img, x, y):
    num_rows, num_cols = img.shape[:2]
    translation_matrix = np.float32([[1, 0, x], [0, 1, y]])
    img_translation = cv2.warpAffine(
        img, translation_matrix, (num_cols, num_rows))
    return img_translation[max(y, 0):num_rows-abs(y), max(x, 0):num_cols-abs(x)]

def add_noise_saltpepper(image, amount):
    row, col, ch = image.shape
    s_vs_p = 0.5
    out = np.copy(image)
    # Salt mode
    num_salt = np.ceil(amount * image.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
    out[coords] = 1
    # Pepper mode
    num_pepper = np.ceil(amount * image.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
    out[coords] = 0
    return out

def add_noise_gaussian(image, mean: float, var: float):
    row, col, ch = image.shape
    sigma = var**0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch)
    noisy = image + gauss
    return noisy

In [19]:
root = '/Volumes/TriveStorage/code/trive-image-recognition/gestures'
classes = [x for x in os.listdir(root) if x != '.DS_Store' and x != 'gauss' and x != 'sp' and x != 'for_training' and x != 'for_training.zip']

gauss_path = os.path.join(root, 'gauss')
sp_path = os.path.join(root, 'sp')
if not os.path.exists(gauss_path):
    os.mkdir(gauss_path)
if not os.path.exists(sp_path):
    os.mkdir(sp_path)

for cl in classes:
    class_path = os.path.join(root, cl)
    
    dst = os.path.join(sp_path, cl)
    if not os.path.exists(dst):
        os.mkdir(dst)
    for image_name in [x for x in os.listdir(class_path) if x != '.DS_Store']:
        try:
            image_dst = os.path.join(dst, image_name)
            if os.path.exists(image_dst):
                continue
            image_path = os.path.join(class_path, image_name)
            cv2.imwrite(image_dst, add_noise_saltpepper(cv2.imread(image_path), 0.01))
        except Exception as e:
            print(e)
    
    dst = os.path.join(gauss_path, cl)
    if not os.path.exists(dst):
        os.mkdir(dst)
    for image_name in [x for x in os.listdir(class_path) if x != '.DS_Store']:
        try:
            image_dst = os.path.join(dst, image_name)
            if os.path.exists(image_dst):
                continue
            image_path = os.path.join(class_path, image_name)
            cv2.imwrite(image_dst, add_noise_gaussian(cv2.imread(image_path), 0, 0.6))
        except Exception as e:
            print(e)



/Users/jo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  app.launch_new_instance()
/Users/jo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
